This notebook uses the cleaned data loaded into the nfl_betting.db to answer key questions about:

1. How many total games are in your dataset?
2. How many games per season?
3. What is the average total points scored across all games?
4. Which season had the highest average total points?
5. What percentage of games did home teams win?
6. Which teams have the most wins as home teams?
7. What is the average margin of victory for home teams vs away teams?
8. How often do favorites win straight up?
9. Which team is the strongest performer as a favorite?
10. Over/Under Hit Rate by Spread Range.
11. Home vs Away Favorite Cover Rate.
12. Build a “Team Power Rating”


In [133]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../data/processed/nfl_betting.db")

In [134]:
pd.read_sql("SELECT * FROM nfl_sports_betting_final ORDER BY schedule_season DESC LIMIT 5;", conn)

,index,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,...,weather_humidity,weather_detail,margin_of_victory,total_points,favorite_is_home,favorite_score,underdog_score,favorite_covered,over_hit,spread_result
0,11597,2025-09-04,2025,1,0,Philadelphia Eagles,24.0,20.0,Dallas Cowboys,PHI,...,66.0,Unknown,4.0,44.0,0,20.0,24.0,0,0,-12.5
1,11598,2025-09-05,2025,1,0,Los Angeles Chargers,27.0,21.0,Kansas City Chiefs,KC,...,80.0,Unknown,6.0,48.0,0,21.0,27.0,0,1,-9.0
2,11599,2025-09-07,2025,1,0,Atlanta Falcons,20.0,23.0,Tampa Bay Buccaneers,ATL,...,68.0,indoor,-3.0,43.0,0,23.0,20.0,1,0,2.0
3,11600,2025-09-07,2025,1,0,Buffalo Bills,41.0,40.0,Baltimore Ravens,BAL,...,61.0,Unknown,1.0,81.0,0,40.0,41.0,0,1,-2.5
4,11601,2025-09-07,2025,1,0,Cleveland Browns,16.0,17.0,Cincinnati Bengals,CIN,...,49.0,Unknown,-1.0,33.0,0,17.0,16.0,0,0,-4.0


In [135]:
games = """
SELECT  
    COUNT(*) AS total_games
FROM 
    nfl_sports_betting_final
ORDER BY total_games
""" 

total_games = pd.read_sql(games, conn)
total_games

,total_games
0,11732


In [136]:
gps = """ 
SELECT
    schedule_season,
    COUNT(schedule_week) AS season_games
FROM 
    nfl_sports_betting_final
GROUP BY schedule_season
ORDER BY season_games DESC
LIMIT 5
"""

games_per_season = pd.read_sql(gps, conn)
games_per_season

,schedule_season,season_games
0,2024,285
1,2023,285
2,2021,285
3,2022,284
4,2020,269


In [137]:
avg_ppg = """ 
SELECT
    AVG(total_points) AS avg_points
FROM
    nfl_sports_betting_final
ORDER BY 
    avg_points
    """

average_points_per_game = pd.read_sql(avg_ppg, conn)
average_points_per_game

,avg_points
0,42.997699


In [138]:
hsa = """ 
SELECT
    schedule_season,
    AVG(total_points) AS avg_points
FROM 
    nfl_sports_betting_final
GROUP BY 
    schedule_season
ORDER BY 
    avg_points DESC
LIMIT 5
"""

highest_season_average_points = pd.read_sql(hsa, conn)
highest_season_average_points

,schedule_season,avg_points
0,1978,66.000000
1,2020,49.490706
2,1967,47.000000
3,2013,46.868914
4,2025,46.711111


In [139]:
hgw = """
SELECT
    schedule_season,
   AVG(CASE WHEN score_home > score_away THEN 1.0 ELSE 0 END) AS home_win_rate,
   COUNT(*) AS total_games
FROM 
    nfl_sports_betting_final
GROUP BY 
    schedule_season
ORDER BY 
    schedule_season DESC
LIMIT 5
"""

home_games_won = pd.read_sql(hgw, conn)
home_games_won

,schedule_season,home_win_rate,total_games
0,2025,0.540741,135
1,2024,0.547368,285
2,2023,0.564912,285
3,2022,0.563380,284
4,2021,0.515789,285


In [140]:
mhtw = """ 
SELECT 
    team_home, 
    COUNT(CASE WHEN score_home > score_away THEN 1.0 ELSE 0 END) AS home_win
FROM 
    nfl_sports_betting_final
GROUP BY 
    team_home
ORDER BY 
    home_win DESC
LIMIT 5    
"""

most_home_team_wins = pd.read_sql(mhtw, conn)
most_home_team_wins

,team_home,home_win
0,San Francisco 49ers,405
1,New England Patriots,400
2,Philadelphia Eagles,399
3,Dallas Cowboys,396
4,Pittsburgh Steelers,395


In [141]:
amov= """
SELECT
    schedule_season,
    AVG(CASE WHEN score_home > score_away THEN 1.0 ELSE 0 END) AS home_margin,
    AVG(CASE WHEN score_home < score_away THEN 1.0 ELSE 0 END) AS away_margin
FROM 
    nfl_sports_betting_final
GROUP BY 
    schedule_season
ORDER BY
    schedule_season DESC
LIMIT 5
"""

average_margin_of_victory = pd.read_sql(amov, conn)
average_margin_of_victory

,schedule_season,home_margin,away_margin
0,2025,0.540741,0.451852
1,2024,0.547368,0.452632
2,2023,0.564912,0.435088
3,2022,0.563380,0.429577
4,2021,0.515789,0.480702


In [142]:
ftw = """ 
SELECT
    schedule_season,
    AVG(CASE WHEN favorite_score > underdog_score THEN 1.0 ELSE 0 END) as favorite_win,
    AVG(CASE WHEN favorite_score < underdog_score THEN 1.0 ELSE 0 END) as underdog_win
FROM 
    nfl_sports_betting_final
GROUP BY 
    schedule_season
ORDER BY 
    schedule_season DESC
LIMIT 5
"""

favorite_team_wins = pd.read_sql(ftw, conn)
favorite_team_wins

,schedule_season,favorite_win,underdog_win
0,2025,0.451852,0.540741
1,2024,0.452632,0.547368
2,2023,0.435088,0.564912
3,2022,0.429577,0.563380
4,2021,0.480702,0.515789


In [143]:
spaf = """
SELECT
    team_home, 
    SUM(CASE WHEN favorite_score > underdog_score THEN 1.0 ELSE 0 END) AS favorite_win,
    COUNT(*) AS favorite_team_wins
FROM 
    nfl_sports_betting_final
GROUP BY
    team_home
ORDER BY 
    favorite_team_wins DESC
LIMIT 5
"""

strongest_performer_as_favorite = pd.read_sql(spaf, conn)
strongest_performer_as_favorite

,team_home,favorite_win,favorite_team_wins
0,San Francisco 49ers,147.0,405
1,New England Patriots,136.0,400
2,Philadelphia Eagles,155.0,399
3,Dallas Cowboys,147.0,396
4,Pittsburgh Steelers,123.0,395


In [ ]:
ouhr = """
SELECT
    spread_result,
    SUM(CASE WHEN over_hit = 1 THEN 1.0 ELSE 0 END) AS over_hit,
    SUM(CASE WHEN over_hit = 0 THEN 2.0 ELSE 0 END) AS under_hit
FROM
    nfl_sports_betting_final
GROUP BY 
    spread_result 
LIMIT 5
"""
over_under_hit_rate = pd.read_sql(ouhr, conn)
over_under_hit_rate

,spread_result,over_hit,under_hit
0,-69.0,1.0,0
1,-68.0,2.0,0
2,-63.0,1.0,0
3,-62.5,1.0,0
4,-61.0,2.0,0
